# Build an Agents system with AutoGen - Author/Reviewer Scenario

## Setup

### Install the required packages

In [ ]:
%pip install autogen-agentchat autogen-ext[openai] click

### Import the required packages

In [ ]:
import os

import click
from dotenv import load_dotenv

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

### Load the environment variables

In [ ]:
load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
MODEL = os.getenv("MODEL", "gpt-4.1")

In [ ]:
print(f"Using endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Using API version: {AZURE_OPENAI_API_VERSION}")
print(f"Using API key: {'*' * len(AZURE_OPENAI_API_KEY)}")
print(f"Using model: {MODEL}")


### Create an OpenAI AutoGen client for Azure

In [ ]:
def get_model_client():
    return AzureOpenAIChatCompletionClient(
            model=MODEL,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
            api_key=AZURE_OPENAI_API_KEY,
            temperature=0.1,
        )
    return instance

### Define the Agents and the conversation termination

In [ ]:
WRITER_SYSTEM_MESSAGE = "You are a AI techical document author. Write a concise document. If revising the document, write the full document with the revisions."
REVIEWER_SYSTEM_MESSAGE = "You are a AI technical document reviewer that can review technical documents. Make sure that your feedback includes edge scenarios if approprite for the topic. Respond with 'APPROVE' when your feedbacks are addressed."

doc_writer = AssistantAgent(
    "doc_writer",
    model_client=get_model_client(),
    system_message=WRITER_SYSTEM_MESSAGE,
)

# Create the critic agent.
doc_reviewer = AssistantAgent(
    "doc_reviewer",
    model_client=get_model_client(),
    system_message=REVIEWER_SYSTEM_MESSAGE,
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

### Configure the Agent conversation pattern

In [ ]:
team = RoundRobinGroupChat(
    [doc_writer, doc_reviewer], termination_condition=text_termination)

### Author and review a technical document

In [ ]:
task = "prompt engineering"

res = team.run_stream(task=task)
# await Console(res)  # Stream the messages to the console.
async for message in res:
    # print(message)
    if isinstance(message, str):
        print("String message:", message)
    elif hasattr(message, "content"):
        if hasattr(message, "source"):
            # print("Message role:", message.role)
            click.echo(click.style(message.source +
                        "\n", fg='cyan', bold=True))
        # print("Message content:", message.content)
        if message.source == "doc_writer":
            click.echo(click.style(message.content + "\n", fg="green"))
        else:
            click.echo(click.style(message.content + "\n", fg="yellow"))
    else:
        # print("Other message type:", message)
        try:
            msg = message.messages[-1]
            # print(msg)
            click.echo(click.style(msg.content + "\n", fg="orange"))
        except Exception as e:
            pass